# PART 1

In [10]:
import open3d as o3d
import numpy as np

velodyne_file = 'D:/Cluster_proj/VIPCClustering/data/KITTI/velodyne/000010.bin'

velo_points = np.fromfile(velodyne_file, dtype=np.float32).reshape(-1, 4)
points_np = velo_points[:, :3]

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points_np)

o3d.visualization.draw_geometries([pcd])


In [11]:
import open3d as o3d
import numpy as np

velodyne_file = 'D:/Cluster_proj/VIPCClustering/data/KITTI/velodyne/000010.bin'

velo_points = np.fromfile(velodyne_file, dtype=np.float32).reshape(-1, 4)
points_np = velo_points[:, :3]

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points_np)

calib_file = 'D:/Cluster_proj/annotator/kitti/data_object/training/calib/000010.txt'

calib_data = {}

with open(calib_file, 'r') as f:
    for line in f:
        if ':' in line:
            key, value = line.strip().split(':')
            value = np.array([float(x) for x in value.strip().split()])
            calib_data[key.strip()] = value

bbox_list = []
for key, value in calib_data.items():
    if key.startswith('P') or key.startswith('R') or key.startswith('Tr'):
        continue
    if len(value) == 12:
        min_bound = value[:3]
        max_bound = value[3:6]
        bbox = o3d.geometry.AxisAlignedBoundingBox(min_bound=min_bound, max_bound=max_bound)
        bbox.color = (0, 0, 0)
        bbox_list.append(bbox)

o3d.visualization.draw_geometries([pcd, *bbox_list])


In [12]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors


velodyne_file = 'D:/Cluster_proj/VIPCClustering/data/KITTI/velodyne/000010.bin'

velo_points = np.fromfile(velodyne_file, dtype=np.float32).reshape(-1, 4)
points_np = velo_points[:, :3]

if len(points_np) == 0:
    print("The point cloud is empty or the file contains no data.")
else:
    scaler = StandardScaler()
    scaled_points_np = scaler.fit_transform(points_np)

    cutting_height = -1.2
    cut_indices = np.where(points_np[:, 2] > cutting_height)[0]
    cut_points_np = points_np[cut_indices]
    cut_scaled_points_np = scaled_points_np[cut_indices]

    n_neighbors = 1

    nbrs = NearestNeighbors(n_neighbors=n_neighbors, algorithm='auto').fit(cut_scaled_points_np)
    distances, indices = nbrs.kneighbors(cut_scaled_points_np)

    avg_distances = np.mean(distances, axis=1)

    threshold_low = np.percentile(avg_distances, 1)

    filtered_indices = np.where(avg_distances <= threshold_low)[0]
    filtered_cut_points_np = cut_points_np[filtered_indices]

    num_clusters = 400
    kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(filtered_cut_points_np)
    labels = kmeans.labels_

    max_label = labels.max()
    colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))

    kmeans_pcd = o3d.geometry.PointCloud()
    kmeans_pcd.points = o3d.utility.Vector3dVector(filtered_cut_points_np)
    kmeans_pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])

    bbox_list = []
    for label in np.unique(labels):
        if label == -1:
            continue
        cluster_points = filtered_cut_points_np[labels == label]
        bbox = o3d.geometry.AxisAlignedBoundingBox.create_from_points(o3d.utility.Vector3dVector(cluster_points))
        bbox.color = (0, 0, 0)
        bbox_list.append(bbox)

    o3d.visualization.draw_geometries([kmeans_pcd] + bbox_list)

    print("Clusters with up to 200 points in each cluster:")
    unique_labels, counts = np.unique(labels, return_counts=True)
    sorted_clusters = sorted(zip(unique_labels, counts), key=lambda x: x[1], reverse=True)
    for cluster_label, point_count in sorted_clusters:
        if point_count <= 200:
            print(f"Cluster {cluster_label}: {point_count} points")
    
filtered_indices = np.ones(len(labels), dtype=bool)
for cluster_label, point_count in sorted_clusters:
    if point_count <= 200:
        filtered_indices[labels == cluster_label] = False

filtered_points_np = filtered_cut_points_np[filtered_indices]

filtered_pcd = o3d.geometry.PointCloud()
filtered_pcd.points = o3d.utility.Vector3dVector(filtered_points_np)

o3d.visualization.draw_geometries([filtered_pcd])


d:\Program\Anaconda\envs\tfenv\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Clusters with up to 200 points in each cluster:
Cluster 150: 200 points
Cluster 230: 195 points
Cluster 175: 194 points
Cluster 350: 191 points
Cluster 35: 187 points
Cluster 73: 187 points
Cluster 68: 185 points
Cluster 51: 184 points
Cluster 300: 184 points
Cluster 244: 182 points
Cluster 107: 180 points
Cluster 115: 178 points
Cluster 185: 177 points
Cluster 95: 174 points
Cluster 296: 173 points
Cluster 104: 171 points
Cluster 165: 171 points
Cluster 242: 170 points
Cluster 96: 168 points
Cluster 103: 167 points
Cluster 59: 162 points
Cluster 153: 157 points
Cluster 58: 156 points
Cluster 380: 155 points
Cluster 295: 154 points
Cluster 32: 152 points
Cluster 42: 149 points
Cluster 194: 149 points
Cluster 55: 148 points
Cluster 386: 148 points
Cluster 274: 146 points
Cluster 3: 144 points
Cluster 247: 144 points
Cluster 337: 143 points
Cluster 155: 142 points
Cluster 101: 140 points
Cluster 352: 135 points
Cluster 11: 134 points
Cluster 251: 132 points
Cluster 287: 131 points
Cluste

In [13]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

points_np = filtered_points_np

if len(points_np) == 0:
    print("Облако точек пустое.")
else:
    scaler = StandardScaler()
    scaled_points_np = scaler.fit_transform(points_np)

    cutting_height = -1.2
    cut_indices = np.where(points_np[:, 2] > cutting_height)[0]
    cut_points_np = points_np[cut_indices]
    cut_scaled_points_np = scaled_points_np[cut_indices]

    n_neighbors = 1

    nbrs = NearestNeighbors(n_neighbors=n_neighbors, algorithm='auto').fit(cut_scaled_points_np)
    distances, indices = nbrs.kneighbors(cut_scaled_points_np)

    avg_distances = np.mean(distances, axis=1)

    threshold_low = np.percentile(avg_distances, 1)

    filtered_indices = np.where(avg_distances <= threshold_low)[0]
    filtered_points_np = cut_points_np[filtered_indices]

    center = np.mean(filtered_points_np, axis=0)

    center_point = o3d.geometry.TriangleMesh.create_sphere(radius=0.1)
    center_point.paint_uniform_color([1.0, 0.0, 0.0])
    center_point.translate(center)

    num_clusters = 15
    kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(filtered_points_np)
    labels = kmeans.labels_

    max_label = labels.max()
    colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))

    kmeans_pcd = o3d.geometry.PointCloud()
    kmeans_pcd.points = o3d.utility.Vector3dVector(filtered_points_np)
    kmeans_pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])

    bbox_list = []
    for label in np.unique(labels):
        if label == -1:
            continue
        cluster_points = filtered_points_np[labels == label]
        bbox = o3d.geometry.AxisAlignedBoundingBox.create_from_points(o3d.utility.Vector3dVector(cluster_points))
        bbox.color = (0, 0, 0)
        bbox_list.append(bbox)

    lines = []
    for cluster_center in kmeans.cluster_centers_:
        line = o3d.geometry.LineSet()
        line.points = o3d.utility.Vector3dVector([center, cluster_center])
        line.lines = o3d.utility.Vector2iVector([[0, 1]])
        line.colors = o3d.utility.Vector3dVector([[0, 0, 0]])
        lines.append(line)

    o3d.visualization.draw_geometries([kmeans_pcd, center_point] + bbox_list + lines)


d:\Program\Anaconda\envs\tfenv\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


# 1

In [14]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

points_np = filtered_points_np

if len(points_np) == 0:
    print("Облако точек пустое.")
else:
    scaler = StandardScaler()
    scaled_points_np = scaler.fit_transform(points_np)

    cutting_height = -1.2
    cut_indices = np.where(points_np[:, 2] > cutting_height)[0]
    cut_points_np = points_np[cut_indices]
    cut_scaled_points_np = scaled_points_np[cut_indices]

    n_neighbors = 1

    nbrs = NearestNeighbors(n_neighbors=n_neighbors, algorithm='auto').fit(cut_scaled_points_np)
    distances, indices = nbrs.kneighbors(cut_scaled_points_np)

    avg_distances = np.mean(distances, axis=1)

    threshold_low = np.percentile(avg_distances, 1)

    filtered_indices = np.where(avg_distances <= threshold_low)[0]
    filtered_points_np = cut_points_np[filtered_indices]

    center = np.mean(filtered_points_np, axis=0)

    center_point = o3d.geometry.TriangleMesh.create_sphere(radius=0.1)
    center_point.paint_uniform_color([1.0, 0.0, 0.0])
    center_point.translate(center)

    num_clusters = 15
    kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(filtered_points_np)
    labels = kmeans.labels_

    lines = []
    points = []
    for cluster_center in kmeans.cluster_centers_:
        line = o3d.geometry.LineSet()
        line.points = o3d.utility.Vector3dVector([center, cluster_center])
        line.lines = o3d.utility.Vector2iVector([[0, 1]])
        line.colors = o3d.utility.Vector3dVector([[0, 0, 0]])
        lines.append(line)
        
        point = o3d.geometry.PointCloud()
        point.points = o3d.utility.Vector3dVector([cluster_center])
        point.paint_uniform_color([1.0, 0.0, 0.0])
        points.append(point)



    o3d.visualization.draw_geometries([center_point] + lines + points)


d:\Program\Anaconda\envs\tfenv\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


# PART 2

In [15]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

points_np = filtered_points_np  # filtered_points_np

if len(points_np) == 0:
    print("The point cloud is empty.")
else:
    scaler = StandardScaler()
    scaled_points_np = scaler.fit_transform(points_np)

    cutting_height = -1.2
    cut_indices = np.where(points_np[:, 2] > cutting_height)[0]
    cut_points_np = points_np[cut_indices]
    cut_scaled_points_np = scaled_points_np[cut_indices]

    n_neighbors = 1

    nbrs = NearestNeighbors(n_neighbors=n_neighbors, algorithm='auto').fit(cut_scaled_points_np)
    distances, indices = nbrs.kneighbors(cut_scaled_points_np)

    avg_distances = np.mean(distances, axis=1)

    threshold_low = np.percentile(avg_distances, 1)

    filtered_indices = np.where(avg_distances <= threshold_low)[0]
    filtered_points_np = cut_points_np[filtered_indices]

    center = np.mean(filtered_points_np, axis=0)

    center_point = o3d.geometry.TriangleMesh.create_sphere(radius=0.1)
    center_point.paint_uniform_color([1.0, 0.0, 0.0])
    center_point.translate(center)

    num_clusters = 15
    kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(filtered_points_np)
    labels = kmeans.labels_

    max_label = labels.max()
    colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))

    kmeans_pcd = o3d.geometry.PointCloud()
    kmeans_pcd.points = o3d.utility.Vector3dVector(filtered_points_np)
    kmeans_pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])

    bbox_list = []
    cluster_sizes = []
    for label in np.unique(labels):
        if label == -1:
            continue
        cluster_points = filtered_points_np[labels == label]
        bbox = o3d.geometry.AxisAlignedBoundingBox.create_from_points(o3d.utility.Vector3dVector(cluster_points))
        bbox.color = (0, 0, 0)
        bbox_list.append(bbox)
        cluster_sizes.append(len(cluster_points))

    lines = []
    for cluster_center in kmeans.cluster_centers_:
        line = o3d.geometry.LineSet()
        line.points = o3d.utility.Vector3dVector([center, cluster_center])
        line.lines = o3d.utility.Vector2iVector([[0, 1]])
        line.colors = o3d.utility.Vector3dVector([[0, 0, 0]])
        lines.append(line)

    o3d.visualization.draw_geometries([kmeans_pcd, center_point] + bbox_list + lines)

    for i, bbox in enumerate(bbox_list, 1):
        print(f"Bounding box {i}:")
        print(f"Center: {bbox.get_center()}")
        print(f"Dimensions: {bbox.get_extent()}")
        print(f"Number of points in a cluster: {cluster_sizes[i-1]}")


d:\Program\Anaconda\envs\tfenv\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Bounding box 1:
Center: [-4.53550005 -3.61149991 -0.7175    ]
Dimensions: [3.40299988 2.00699973 0.963     ]
Number of points in a cluster: 2165
Bounding box 2:
Center: [  2.54600006 -15.85900021  -0.2015    ]
Dimensions: [4.05000007 4.00399971 1.995     ]
Number of points in a cluster: 1406
Bounding box 3:
Center: [ 1.64599996  3.62149988 -0.6435    ]
Dimensions: [3.02599992 6.32899979 1.111     ]
Number of points in a cluster: 2941
Bounding box 4:
Center: [-9.42749977  7.47700024 -0.86500001]
Dimensions: [1.61499977 1.3920002  0.66400003]
Number of points in a cluster: 218
Bounding box 5:
Center: [16.5619998  -5.63849998 -0.71449998]
Dimensions: [2.80000019 1.44700003 0.96699995]
Number of points in a cluster: 251
Bounding box 6:
Center: [ -4.35450006 -12.64999962  -0.2385    ]
Dimensions: [3.4749999  4.01399994 1.921     ]
Number of points in a cluster: 2178
Bounding box 7:
Center: [ -1.05599999 -13.69000006  -0.24449998]
Dimensions: [3.19799995 2.29799938 1.90699995]
Number of poin

In [16]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
# import os

points_np = filtered_points_np  # filtered_points_np должна быть определена заранее

if len(points_np) == 0:
    print("Облако точек пустое.")
else:
    scaler = StandardScaler()
    scaled_points_np = scaler.fit_transform(points_np)

    cutting_height = -1.2
    cut_indices = np.where(points_np[:, 2] > cutting_height)[0]
    cut_points_np = points_np[cut_indices]
    cut_scaled_points_np = scaled_points_np[cut_indices]

    n_neighbors = 1

    nbrs = NearestNeighbors(n_neighbors=n_neighbors, algorithm='auto').fit(cut_scaled_points_np)
    distances, indices = nbrs.kneighbors(cut_scaled_points_np)

    avg_distances = np.mean(distances, axis=1)

    threshold_low = np.percentile(avg_distances, 1)

    filtered_indices = np.where(avg_distances <= threshold_low)[0]
    filtered_points_np = cut_points_np[filtered_indices]

    center = np.mean(filtered_points_np, axis=0)

    center_point = o3d.geometry.TriangleMesh.create_sphere(radius=0.1)
    center_point.paint_uniform_color([1.0, 0.0, 0.0])
    center_point.translate(center)

    num_clusters = 15
    kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(filtered_points_np)
    labels = kmeans.labels_

    max_label = labels.max()
    colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))

    kmeans_pcd = o3d.geometry.PointCloud()
    kmeans_pcd.points = o3d.utility.Vector3dVector(filtered_points_np)
    kmeans_pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])

    bbox_list = []
    cluster_sizes = []
    for label in np.unique(labels):
        if label == -1:
            continue
        cluster_points = filtered_points_np[labels == label]
        bbox = o3d.geometry.AxisAlignedBoundingBox.create_from_points(o3d.utility.Vector3dVector(cluster_points))
        bbox.color = (0, 0, 0)
        bbox_list.append(bbox)
        cluster_sizes.append(len(cluster_points))

    lines = []
    for cluster_center in kmeans.cluster_centers_:
        line = o3d.geometry.LineSet()
        line.points = o3d.utility.Vector3dVector([center, cluster_center])
        line.lines = o3d.utility.Vector2iVector([[0, 1]])
        line.colors = o3d.utility.Vector3dVector([[0, 0, 0]])
        lines.append(line)

    o3d.visualization.draw_geometries([kmeans_pcd, center_point] + bbox_list + lines)


d:\Program\Anaconda\envs\tfenv\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [17]:
import numpy as np
import os

for i, center_coord in enumerate(kmeans.cluster_centers_):
    distance_to_center = np.linalg.norm(center_coord)
    print(f"Cluster center {i}: {center_coord}, Distance to the center of coordinates: {distance_to_center}")

output_dir = r'D:\Cluster_proj\VIPCClustering\KITTI_clusters\experiment_000010\cluster'

os.makedirs(output_dir, exist_ok=True)

output_file_path = os.path.join(output_dir, "bbox_info.txt")

with open(output_file_path, "w", encoding="utf-8") as file:
    for i, bbox in enumerate(bbox_list, 1):
        file.write(f"bounding box {i}:\n")
        file.write(f"center: {bbox.get_center().tolist()}\n")
        file.write(f"Dimensions: {bbox.get_extent().tolist()}\n")
        file.write(f"Number of points in a cluster: {cluster_sizes[i-1]}\n")
        file.write(f"Distance to the center of coordinates: {np.linalg.norm(bbox.get_center())}\n")

        print(f"bounding box {i}:")
        print(f"center: {bbox.get_center()}")
        print(f"Dimensions: {bbox.get_extent()}")
        print(f"Number of points in a cluster: {cluster_sizes[i-1]}")
        print(f"Distance to the center of coordinates: {np.linalg.norm(bbox.get_center())}")

print(f"Save to: {output_file_path}")


Cluster center 0: [-3.8713398  -3.2828016  -0.88498604], Distance to the center of coordinates: 5.152403354644775
Cluster center 1: [  2.188631   -15.751724    -0.23991439], Distance to the center of coordinates: 15.90485668182373
Cluster center 2: [ 1.9504266  4.57518   -0.8449435], Distance to the center of coordinates: 5.044835567474365
Cluster center 3: [-8.982605    7.3753433  -0.90766513], Distance to the center of coordinates: 11.657904624938965
Cluster center 4: [15.530764   -5.5644464  -0.76592433], Distance to the center of coordinates: 16.515275955200195
Cluster center 5: [ -4.172969  -11.924452   -0.2812395], Distance to the center of coordinates: 12.636665344238281
Cluster center 6: [ -1.1393702 -13.85027    -0.2342656], Distance to the center of coordinates: 13.899030685424805
Cluster center 7: [ 4.1119    -4.0379577 -0.8594338], Distance to the center of coordinates: 5.826786994934082
Cluster center 8: [-23.648388   10.2114105   0.3779717], Distance to the center of coor

# PART 3

In [18]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from scipy.spatial.distance import pdist

def dunn_index(X, labels):
    # Check if the number of data points matches the number of labels
    if len(X) != len(labels):
        raise ValueError("Number of data points and labels do not match.")

    # Check if there are enough unique labels to form clusters
    unique_labels = np.unique(labels)
    if len(unique_labels) < 2:
        raise ValueError("At least two different clusters are required.")

    # Compute intra-cluster distances
    intra_cluster_distances = []
    for label in unique_labels:
        cluster_points = X[labels == label]
        if len(cluster_points) <= 1:
            intra_cluster_distances.append(0)  # Not enough data to assess intra-cluster distances
        else:
            intra_cluster_distances.append(np.max(pdist(cluster_points)))

    # Compute inter-cluster distances
    min_inter_cluster_distance = np.inf
    for i in range(len(unique_labels)):
        for j in range(i + 1, len(unique_labels)):
            cluster_points_i = X[labels == unique_labels[i]]
            cluster_points_j = X[labels == unique_labels[j]]
            inter_cluster_distance = np.min(pdist(np.vstack([cluster_points_i, cluster_points_j])))
            if inter_cluster_distance < min_inter_cluster_distance:
                min_inter_cluster_distance = inter_cluster_distance

    # Compute Dunn Index
    dunn_index_value = min_inter_cluster_distance / np.max(intra_cluster_distances)
    return dunn_index_value

points_np = filtered_points_np

if len(points_np) == 0:
    print("Облако точек пустое.")
else:
    scaler = StandardScaler()
    scaled_points_np = scaler.fit_transform(points_np)

    cutting_height = -1.2
    cut_indices = np.where(points_np[:, 2] > cutting_height)[0]
    cut_points_np = points_np[cut_indices]
    cut_scaled_points_np = scaled_points_np[cut_indices]

    n_neighbors = 1

    nbrs = NearestNeighbors(n_neighbors=n_neighbors, algorithm='auto').fit(cut_scaled_points_np)
    distances, indices = nbrs.kneighbors(cut_scaled_points_np)

    avg_distances = np.mean(distances, axis=1)

    threshold_low = np.percentile(avg_distances, 1)

    filtered_indices = np.where(avg_distances <= threshold_low)[0]
    filtered_points_np = cut_points_np[filtered_indices]

    center = np.mean(filtered_points_np, axis=0)

    center_point = o3d.geometry.TriangleMesh.create_sphere(radius=0.1)
    center_point.paint_uniform_color([1.0, 0.0, 0.0])
    center_point.translate(center)

    num_clusters = 15
    kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(filtered_points_np)
    labels = kmeans.labels_

    silhouette_score_value = silhouette_score(filtered_points_np, labels)
    print("Silhouette Score:", silhouette_score_value)

    try:
        dunn_index_value = dunn_index(filtered_points_np, labels)
        print("Dunn Index:", dunn_index_value)
    except ValueError as e:
        print("Error:", e)

    davies_bouldin_value = davies_bouldin_score(filtered_points_np, labels)
    print("Davies-Bouldin Index:", davies_bouldin_value)

    calinski_harabasz_value = calinski_harabasz_score(filtered_points_np, labels)
    print("Calinski–Harabasz Index:", calinski_harabasz_value)
    calinski_harabasz_value = calinski_harabasz_score(filtered_points_np, labels)
    print("Calinski–Harabasz Index:", calinski_harabasz_value)

    c_index = calinski_harabasz_value / davies_bouldin_value
    print("C-index:", c_index)


    max_label = labels.max()
    colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))

    kmeans_pcd = o3d.geometry.PointCloud()
    kmeans_pcd.points = o3d.utility.Vector3dVector(filtered_points_np)
    kmeans_pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])

    bbox_list = []
    for label in np.unique(labels):
        if label == -1:
            continue
        cluster_points = filtered_points_np[labels == label]
        bbox = o3d.geometry.AxisAlignedBoundingBox.create_from_points(o3d.utility.Vector3dVector(cluster_points))
        bbox.color = (0, 0, 0)
        bbox_list.append(bbox)

    lines = []
    for cluster_center in kmeans.cluster_centers_:
        line = o3d.geometry.LineSet()
        line.points = o3d.utility.Vector3dVector([center, cluster_center])
        line.lines = o3d.utility.Vector2iVector([[0, 1]])
        line.colors = o3d.utility.Vector3dVector([[0, 0, 0]])
        lines.append(line)

    o3d.visualization.draw_geometries([kmeans_pcd, center_point] + bbox_list + lines)



d:\Program\Anaconda\envs\tfenv\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Silhouette Score: 0.71279114
Dunn Index: 0.0009633907613852348
Davies-Bouldin Index: 0.30853531137098905
Calinski–Harabasz Index: 98050.39619632333
Calinski–Harabasz Index: 98050.39619632333
C-index: 317793.1101650325


In [19]:
def variety_index(X, labels):
    clusters = np.unique(labels)
    diversity_per_cluster = []
    for cluster in clusters:
        cluster_points = X[labels == cluster]
        if len(cluster_points) <= 1:
            diversity_per_cluster.append(0)
        else:
            distances = np.linalg.norm(cluster_points[:, None] - cluster_points, axis=-1)
            diversity_per_cluster.append(np.std(distances))
    return np.mean(diversity_per_cluster)

variety_index_value = variety_index(filtered_points_np, labels)
print("Variety Index:", variety_index_value)


Variety Index: 0.57030874


In [20]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from scipy.spatial.distance import pdist

def dunn_index(X, labels):
    # Check if the number of data points matches the number of labels
    if len(X) != len(labels):
        raise ValueError("Number of data points and labels do not match.")

    # Check if there are enough unique labels to form clusters
    unique_labels = np.unique(labels)
    if len(unique_labels) < 2:
        raise ValueError("At least two different clusters are required.")

    # Compute intra-cluster distances
    intra_cluster_distances = []
    for label in unique_labels:
        cluster_points = X[labels == label]
        if len(cluster_points) <= 1:
            intra_cluster_distances.append(0)  # Not enough data to assess intra-cluster distances
        else:
            intra_cluster_distances.append(np.max(pdist(cluster_points)))

    min_inter_cluster_distance = np.inf
    for i in range(len(unique_labels)):
        for j in range(i + 1, len(unique_labels)):
            cluster_points_i = X[labels == unique_labels[i]]
            cluster_points_j = X[labels == unique_labels[j]]
            inter_cluster_distance = np.min(pdist(np.vstack([cluster_points_i, cluster_points_j])))
            if inter_cluster_distance < min_inter_cluster_distance:
                min_inter_cluster_distance = inter_cluster_distance

    dunn_index_value = min_inter_cluster_distance / np.max(intra_cluster_distances)
    return dunn_index_value

points_np = filtered_points_np

if len(points_np) == 0:
    print("Облако точек пустое.")
else:
    scaler = StandardScaler()
    scaled_points_np = scaler.fit_transform(points_np)

    cutting_height = -1.2
    cut_indices = np.where(points_np[:, 2] > cutting_height)[0]
    cut_points_np = points_np[cut_indices]
    cut_scaled_points_np = scaled_points_np[cut_indices]

    n_neighbors = 1

    nbrs = NearestNeighbors(n_neighbors=n_neighbors, algorithm='auto').fit(cut_scaled_points_np)
    distances, indices = nbrs.kneighbors(cut_scaled_points_np)

    avg_distances = np.mean(distances, axis=1)

    threshold_low = np.percentile(avg_distances, 1)

    filtered_indices = np.where(avg_distances <= threshold_low)[0]
    filtered_points_np = cut_points_np[filtered_indices]

    center = np.mean(filtered_points_np, axis=0)

    center_point = o3d.geometry.TriangleMesh.create_sphere(radius=0.1)
    center_point.paint_uniform_color([1.0, 0.0, 0.0])
    center_point.translate(center)

    num_clusters = 15
    kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(filtered_points_np)
    labels = kmeans.labels_

    silhouette_score_value = silhouette_score(filtered_points_np, labels)
    print("Silhouette Score:", silhouette_score_value)

    try:
        dunn_index_value = dunn_index(filtered_points_np, labels)
        print("Dunn Index:", dunn_index_value)
    except ValueError as e:
        print("Error:", e)

    davies_bouldin_value = davies_bouldin_score(filtered_points_np, labels)
    print("Davies-Bouldin Index:", davies_bouldin_value)

    calinski_harabasz_value = calinski_harabasz_score(filtered_points_np, labels)
    print("Calinski–Harabasz Index:", calinski_harabasz_value)
    calinski_harabasz_value = calinski_harabasz_score(filtered_points_np, labels)
    print("Calinski–Harabasz Index:", calinski_harabasz_value)

    c_index = calinski_harabasz_value / davies_bouldin_value
    print("C-index:", c_index)


    max_label = labels.max()
    colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))

    kmeans_pcd = o3d.geometry.PointCloud()
    kmeans_pcd.points = o3d.utility.Vector3dVector(filtered_points_np)
    kmeans_pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])

    bbox_list = []
    for label in np.unique(labels):
        if label == -1:
            continue
        cluster_points = filtered_points_np[labels == label]
        bbox = o3d.geometry.AxisAlignedBoundingBox.create_from_points(o3d.utility.Vector3dVector(cluster_points))
        bbox.color = (0, 0, 0)
        bbox_list.append(bbox)

    lines = []
    for cluster_center in kmeans.cluster_centers_:
        line = o3d.geometry.LineSet()
        line.points = o3d.utility.Vector3dVector([center, cluster_center])
        line.lines = o3d.utility.Vector2iVector([[0, 1]])
        line.colors = o3d.utility.Vector3dVector([[0, 0, 0]])
        lines.append(line)

    o3d.visualization.draw_geometries([kmeans_pcd, center_point] + bbox_list + lines)

    # Calculate distance from origin to each cluster center
    origin = np.array([0, 0, 0])
    cluster_centers = kmeans.cluster_centers_
    distances_to_origin = np.linalg.norm(cluster_centers - origin, axis=1)

    # Calculate the number of points in each cluster
    points_per_cluster = np.bincount(labels)

    # Print the results
    for i, (distance, num_points) in enumerate(zip(distances_to_origin, points_per_cluster)):
        print(f"Cluster {i}: Distance to origin = {distance:.2f}, Number of points = {num_points}")


d:\Program\Anaconda\envs\tfenv\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Silhouette Score: 0.71279114
Dunn Index: 0.0009633907613852348
Davies-Bouldin Index: 0.30853531137098905
Calinski–Harabasz Index: 98050.39619632333
Calinski–Harabasz Index: 98050.39619632333
C-index: 317793.1101650325
Cluster 0: Distance to origin = 5.15, Number of points = 2165
Cluster 1: Distance to origin = 15.90, Number of points = 1406
Cluster 2: Distance to origin = 5.04, Number of points = 2941
Cluster 3: Distance to origin = 11.66, Number of points = 218
Cluster 4: Distance to origin = 16.52, Number of points = 251
Cluster 5: Distance to origin = 12.64, Number of points = 2178
Cluster 6: Distance to origin = 13.90, Number of points = 1764
Cluster 7: Distance to origin = 5.83, Number of points = 1042
Cluster 8: Distance to origin = 25.76, Number of points = 214
Cluster 9: Distance to origin = 11.54, Number of points = 548
Cluster 10: Distance to origin = 16.78, Number of points = 293
Cluster 11: Distance to origin = 11.57, Number of points = 670
Cluster 12: Distance to origin = 

In [21]:
import os

output_dir = r'D:\Cluster_proj\VIPCClustering\KITTI_clusters\experiment_000010\cluster'

os.makedirs(output_dir, exist_ok=True)

output_file_path = os.path.join(output_dir, "indexes.txt")

with open(output_file_path, "w", encoding="utf-8") as file:
    file.write(f"Silhouette Score: {silhouette_score_value}\n")
    file.write(f"Dunn Index: {dunn_index_value}\n")
    file.write(f"Davies-Bouldin Index: {davies_bouldin_value}\n")
    file.write(f"Calinski–Harabasz Index: {calinski_harabasz_value}\n")
    file.write(f"C-index: {c_index}\n")

print(f"Save to: {output_file_path}")


Save to: D:\Cluster_proj\VIPCClustering\KITTI_clusters\experiment_000010\cluster\indexes.txt


In [22]:
import open3d as o3d
import numpy as np

velodyne_file = 'D:/Cluster_proj/VIPCClustering/data/KITTI/velodyne/000010.bin'

velo_points = np.fromfile(velodyne_file, dtype=np.float32).reshape(-1, 4)
points_np = velo_points[:, :3]

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points_np)

labels_file = 'D:/Cluster_proj/annotator/kitti/data_object/training/gt_segmentation/000010.txt'
with open(labels_file, 'r') as file:
    labels = file.readlines()

colors = []
for label in labels:
    if 'BG' in label:
        colors.append([1, 0, 0])
    elif 'Car' in label:
        colors.append([0, 0, 0])

pcd.colors = o3d.utility.Vector3dVector(colors)

o3d.visualization.draw_geometries([pcd])


In [23]:
import open3d as o3d
import numpy as np

velodyne_file = 'D:/Cluster_proj/VIPCClustering/data/KITTI/velodyne/000010.bin'

velo_points = np.fromfile(velodyne_file, dtype=np.float32).reshape(-1, 4)
points_np = velo_points[:, :3]

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points_np)

labels_file = 'D:/Cluster_proj/annotator/kitti/data_object/training/gt_segmentation/000010.txt'
with open(labels_file, 'r') as file:
    labels = file.readlines()

car_indices = []
for i, label in enumerate(labels):
    if 'Car' in label:
        car_indices.append(i)

colors = [[1, 0, 1]] * len(points_np)
for idx in car_indices:
    colors[idx] = [0, 0, 0]

pcd.colors = o3d.utility.Vector3dVector(colors)

o3d.visualization.draw_geometries([pcd])
